<a href="https://colab.research.google.com/github/chengwenn00/GPUDashboard/blob/main/llama3_FYP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies


In [1]:
!pip install -r requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.6 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0


In [4]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline


HF account COnfiguration

In [6]:
config_data = json.load(open("config.json"))
HF_TOKEN = config_data["HF_TOKEN"]


In [8]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

""Quantisation COnfiguration""

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [12]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128
)

In [13]:
def get_response(prompt):
  sequences = text_generator(prompt)
  gen_text = sequences[0]["generated_text"]
  return gen_text

In [34]:
prompt = """
You are a drone tasked with navigating a space. Here is an example of how to provide the steps, ensuring proper spacing and clarity:

Example:
1. Fly forward 5 meters.
2. Turn slightly to the right.
3. Fly forward another 5 meters.

Now, provide the necessary steps for the following instructions:
1. Fly 10 meters towards the tree.
2. Avoid the rock that is 5 meters ahead on the left.

Please format the response clearly with one step per line, and make sure to list all steps.

Your response should look like this:

1.
2.
3.

"""


In [35]:
llama3_response = get_response(prompt)

In [36]:
llama3_response

"\nYou are a drone tasked with navigating a space. Here is an example of how to provide the steps, ensuring proper spacing and clarity:\n\nExample:\n1. Fly forward 5 meters.\n2. Turn slightly to the right.\n3. Fly forward another 5 meters.\n\nNow, provide the necessary steps for the following instructions:\n1. Fly 10 meters towards the tree.\n2. Avoid the rock that is 5 meters ahead on the left.\n\nPlease format the response clearly with one step per line, and make sure to list all steps.\n\nYour response should look like this:\n\n1.\n2.\n3.\n\nSteps:\n1. Fly 10 meters towards the tree.\n2. Avoid the rock that is 5 meters ahead on the left.\n3. Continue flying forward, maintaining a safe distance from the rock.\n4. When you reach the tree, fly around it to the right to ensure a clear path.\n5. Proceed with caution, keeping an eye on the surroundings to avoid any other obstacles.\n\nNote: The original instructions are not necessarily the most efficient or safe, but they are provided as 

In [37]:
print(llama3_response[len(prompt):])

Steps:
1. Fly 10 meters towards the tree.
2. Avoid the rock that is 5 meters ahead on the left.
3. Continue flying forward, maintaining a safe distance from the rock.
4. When you reach the tree, fly around it to the right to ensure a clear path.
5. Proceed with caution, keeping an eye on the surroundings to avoid any other obstacles.

Note: The original instructions are not necessarily the most efficient or safe, but they are provided as a starting point for the drone's navigation. In a real-world scenario, the drone would likely use more advanced navigation and obstacle avoidance systems. 

Please
